## Load Packages

In [1]:
include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

# Mean-Variance: Brute Force

In [3]:
μ = [0.115; 0.095; 0.06]        #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  

println("expected returns: ")
printmat(μ)
println("covariance matrix:")
printmat(Σ)

expected returns: 
     0.115
     0.095
     0.060

covariance matrix:
     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010



To calculate the expeted return and the variance of a portfolio, use $ER_p = w'\mu$ and $Var(R_p) = w'\Sigma w$.

In [4]:
w = [ 0    1    0;
      0.22 0.30 0.48;
      0.02 0.63 0.35;
      0.25 0.68 0.07]'        #4 different portfolios (try many...), notice '

println("Several different portfolios (one in each column): ")
printmat(w)

n = size(w,2)
ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    ERp[i]   = (w[:,i]'μ)[1]
    StdRp[i] = sqrt((w[:,i]'Σ*w[:,i])[1])
end    
println("expected portfolio return and Std of portfolio return for $n portfolios, %: ")
printmat([ERp StdRp]*100)

Several different portfolios (one in each column): 
     0.000     0.220     0.020     0.250
     1.000     0.300     0.630     0.680
     0.000     0.480     0.350     0.070

expected portfolio return and Std of portfolio return for 4 portfolios, %: 
     9.500     8.000
     8.260     7.398
     8.315     6.415
     9.755     7.378



In [5]:
plot1 = scatter(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# MV Frontier with Two Assets

Recall: with only two investable assets, all portfolios of them are on the MV frontier

In [6]:
μ = [0.115; 0.06]            #means
Σ  = [166   58;              #covariance matrix
       58  100]/100^2                      

w1 = collect(1:-0.05:0)            #weight on asset 1 
n  = length(w1)
ERp   = fill(NaN,n)
StdRp = fill(NaN,n)
for i = 1:n
    w        = [w1[i];1-w1[i]]
    ERp[i]   = (w'μ)[1]
    StdRp[i] = sqrt((w'Σ*w)[1])
end   

plot2 = plot(StdRp*100,ERp*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, %

# Calculating the MV Frontier (of only Risky Assets)

All portfolio on the MV frontier of risky assets only have (vectors of) portfolio weights as in

$w = \Sigma^{-1}(\mu \lambda + \mathbf{1} \delta)$, 

where $\lambda$ and $\delta$ depends on $(\mu,\Sigma)$ and the required average return $\mu^*$

In [7]:
μ = [0.115; 0.095; 0.06]    #expected returns
Σ  = [166  34  58;              #covariance matrix
       34  64   4;
       58   4 100]/100^2                  
Rf = 0.03

println("μ, Σ and Rf: ")
printmat(μ)
printmat(Σ)
printmat(Rf)

μ, Σ and Rf: 
     0.115
     0.095
     0.060

     0.017     0.003     0.006
     0.003     0.006     0.000
     0.006     0.000     0.010

     0.030



Some of the intermediate calculations are as follows. (Later on we put it all in a function)

In [8]:
mustar = 0.1                          #required average return  

n    = length(μ)
oneV = ones(n,1) 
 
Σ_1  = inv(Σ)

println("inv(Σ):")
printmat(Σ_1)
println("Σ*inv(Σ):")
printmat(Σ*Σ_1)

A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
B    = (μ'Σ_1*oneV)[1]
C    = (oneV'Σ_1*oneV)[1]
printlnPs("A, B, C: ",[A B C ])

λ    = (C*mustar - B)/(A*C-B^2)
δ    = (A-B*mustar)/(A*C-B^2)
printlnPs("λ and δ: ",[λ δ])

w    = Σ_1 *(μ*λ + oneV*δ)
println("w: ")
printmat(w)

inv(Σ):
    85.735   -42.545   -48.024
   -42.545   177.754    17.566
   -48.024    17.566   127.151

Σ*inv(Σ):
     1.000     0.000     0.000
     0.000     1.000     0.000
    -0.000     0.000     1.000

A, B, C:      1.804    19.759   244.634
λ and δ:      0.093    -0.003
w: 
     0.288
     0.690
     0.022



In [9]:
function MVCalc(mustar,μ,Σ)           #calculates the std of a portfolio on MVF of risky assets
    n    = length(μ)
    oneV = ones(n,1) 
    Σ_1  = inv(Σ)
    A    = (μ'Σ_1*μ)[1]                  #()[1] to make it a scalar, not an 1x1 matrix
    B    = (μ'Σ_1*oneV)[1]
    C    = (oneV'Σ_1*oneV)[1]
    λ    = (C*mustar - B)/(A*C-B^2)
    δ    = (A-B*mustar)/(A*C-B^2)
    w    = Σ_1 *(μ*λ + oneV*δ)
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalc (generic function with 1 method)

In [10]:
(StdAt10,wAt10) = MVCalc(0.1,μ,Σ)
println("Just testing: std and w of portfolio with mean return 10: ", round(StdAt10,3))
printmat(wAt10)

Just testing: std and w of portfolio with mean return 10: 0.077
     0.288
     0.690
     0.022



In [11]:
mustar = linspace(Rf,0.15,101)
L      = length(mustar)
StdRp  = fill(NaN,L)
for i = 1:L
    StdRP_i, = MVCalc(mustar[i],μ,Σ)             #, = means skipping the remaining output
    StdRp[i] = StdRP_i
end    

plot3 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>

# Calculating the MV Frontier (of Risky and Riskfree Assets)

All portfolios on the MV frontier of both risky and riskfree have (vectors of) portfolio weights on the risky assets as in 

$w=\frac{\mu^{\ast}-R_{f}}{(\mu^{e})^{\prime}\Sigma^{-1}\mu^{e}}\Sigma^{-1}
\mu^{e}$, 

where $\mu^*$ is the required average return.

The weight of the riskfree asset is $1-\mathbf{1}'w$

In [12]:
function MVCalcRf(mustar,μ,Σ,Rf)           #calculates the std of a portfolio on MVF of (Risky,Riskfree)
    n     = length(μ)
    μe    = μ - Rf                         #expected excess returns            
    Σ_1   = inv(Σ)
    w     = (mustar-Rf)/(μe'Σ_1*μe)[1] * Σ_1 *μe
    StdRp = sqrt((w'Σ*w)[1])
    return StdRp,w
end

MVCalcRf (generic function with 1 method)

In [13]:
StdRpRf  = fill(NaN,L)
for i = 1:L
    StdRpRf_i, = MVCalcRf(mustar[i],μ,Σ,Rf) 
    StdRpRf[i] = StdRpRf_i
end    

plot4 = plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(sqrt.(diag(Σ))*100,μ*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,300.416 59.4998,297.645 64.2293,294.875 68.9587,292.105 73.6882,289.334 78.4176,286.564 83.1471,283.794 87.8766,281.023 92.606,278.253 97.3355,275.483 
 102.065,272.712 106.794,269.942 111.524,267.171 116.253,264.401 120.983,261.631 125.712,258.86 130.442,256.09 135.171,253.32 139.901,250.549 144.63,247.779 
 149.36,245.009 154.089,242.238 158.818,239.468 163.548,236.698 168.277,233.927 173.007,231.157 177.736,228.387 182.466,225.616 187.195,222.846 191.925,220.076 
 196.654,217.305 201.384,214.535 206.113,211.765 210.843,208.994 215.572,206.224 220.301,203.454 225.031,200.683 229.76,197.913 234.49,195.143 239.219,192.372 
 243.949,189.602 248.678,186.832 253.408,184.061 258.137,181.291 262.867,178.521 267.596,175.75 272.326,172.98 277.055,170.21 281.784,167.439 286.514,164.669 
 291.243,161.899 295.973,159.128 300.702,156.358 305.432,153.587 310.161,150.817 314.891,148.047 319.62,145.276 324.35,142.506 329.079,139.736 333.809,136.965 
 338.538,134.195 343.267,131.425 347.997,128.654 352.726,125.884 357.456,123.114 362.185,120.343 366.915,117.573 371.644,114.803 376.374,112.032 381.103,109.262 
 385.833,106.492 390.562,103.721 395.292,100.951 400.021,98.1807 404.751,95.4103 409.48,92.64 414.209,89.8696 418.939,87.0993 423.668,84.329 428.398,81.5586 
 433.127,78.7883 437.857,76.0179 442.586,73.2476 447.316,70.4773 452.045,67.7069 456.775,64.9366 461.504,62.1662 466.234,59.3959 470.963,56.6255 475.692,53.8552 
 480.422,51.0849 485.151,48.3145 489.881,45.5442 494.61,42.7738 499.34,40.0035 504.069,37.2332 508.799,34.4628 513.528,31.6925 518.258,28.9221 522.987,26.1518 
 527.717,23.3815 
 "/>

# Tangency Portfolio

The tangency portfolio is a particular portfolio on the MV frontier of risky and riskfree, where the weights on the risky assets sum to one. It is therefore also on the MV frontier of risky assets only. The vector of portfolio weights is

$w_{T}=\frac{\Sigma^{-1}\mu^{e}}{\mathbf{1}^{\prime}\Sigma^{-1}\mu^{e}}$

In [14]:
function MVTangencyP(μ,Σ,Rf)           #calculates the tangency portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     #expected excess returns            
    Σ_1   = inv(Σ)
    w     = Σ_1 *μe/(oneV'Σ_1*μe)[1]
    muT   = (w'μ + (1-sum(w))*Rf)[1]
    StdT = sqrt((w'Σ*w)[1])
    return w,muT,StdT
end

MVTangencyP (generic function with 1 method)

In [15]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
println("Tangency portfolio: ")
printmat(wT)
printlnPs("mean and std of tangency portfolio, %: ",[muT StdT]*100)

Tangency portfolio: 
     0.248
     0.682
     0.070

mean and std of tangency portfolio, %:      9.750     7.372


By mixing the tangency portfolio and the riskfree, we can create any point on the MV frontier (for risky and riskfree). 

The code below shows the expected return and standard deviation of several portfolio (different $v$ values) of the form

$R_p = v w_T'R + (1-v)R_f$ 

In [16]:
v = [0;0.44;1;1.41]                                 #try different mixes of wT and Rf

(ERLev,StdLev)  = (similar(v),similar(v))           #to store results in
for i = 1:length(v)
    w          = v[i]*wT                          #risky portfolio
    ERLev[i]   = (w'μ + (1-v[i])*Rf)[1]     #portfolio with v[i] in wT and 1-v[i] in Rf
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,300.416 59.4998,297.645 64.2293,294.875 68.9587,292.105 73.6882,289.334 78.4176,286.564 83.1471,283.794 87.8766,281.023 92.606,278.253 97.3355,275.483 
 102.065,272.712 106.794,269.942 111.524,267.171 116.253,264.401 120.983,261.631 125.712,258.86 130.442,256.09 135.171,253.32 139.901,250.549 144.63,247.779 
 149.36,245.009 154.089,242.238 158.818,239.468 163.548,236.698 168.277,233.927 173.007,231.157 177.736,228.387 182.466,225.616 187.195,222.846 191.925,220.076 
 196.654,217.305 201.384,214.535 206.113,211.765 210.843,208.994 215.572,206.224 220.301,203.454 225.031,200.683 229.76,197.913 234.49,195.143 239.219,192.372 
 243.949,189.602 248.678,186.832 253.408,184.061 258.137,181.291 262.867,178.521 267.596,175.75 272.326,172.98 277.055,170.21 281.784,167.439 286.514,164.669 
 291.243,161.899 295.973,159.128 300.702,156.358 305.432,153.587 310.161,150.817 314.891,148.047 319.62,145.276 324.35,142.506 329.079,139.736 333.809,136.965 
 338.538,134.195 343.267,131.425 347.997,128.654 352.726,125.884 357.456,123.114 362.185,120.343 366.915,117.573 371.644,114.803 376.374,112.032 381.103,109.262 
 385.833,106.492 390.562,103.721 395.292,100.951 400.021,98.1807 404.751,95.4103 409.48,92.64 414.209,89.8696 418.939,87.0993 423.668,84.329 428.398,81.5586 
 433.127,78.7883 437.857,76.0179 442.586,73.2476 447.316,70.4773 452.045,67.7069 456.775,64.9366 461.504,62.1662 466.234,59.3959 470.963,56.6255 475.692,53.8552 
 480.422,51.0849 485.151,48.3145 489.881,45.5442 494.61,42.7738 499.34,40.0035 504.069,37.2332 508.799,34.4628 513.528,31.6925 518.258,28.9221 522.987,26.1518 
 527.717,23.3815 
 "/>

## Examples of Tangency Portfolios

In [17]:
μb = [9; 6]/100                     #means
Σb = [ 256  0;
      0    144]/100^2
Rfb = 1/100
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

wT, = MVTangencyP([13; 6]/100,Σb,Rfb)
printmat(wT)

Σb = [ 1  -0.8;
      -0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

Σb = [ 1  0.8;
      0.8    1]
wT, = MVTangencyP(μb,Σb,Rfb)
printmat(wT)

     0.474
     0.526

     0.574
     0.426

     0.513
     0.487

     1.538
    -0.538



# Drawing the MV Frontier (Risky Assets Only) Revisisted (extra)

Recall: with only two investable assets, all portfolios of them are on the MV frontier.

We apply this idea by using the global minimum variance portfolio (see below for code) and the tangency portfolios (see above).

In [18]:
function MVMinimumVarP(μ,Σ,Rf)           #calculates the global minimum variance portfolio
    n     = length(μ)
    oneV  = ones(n,1)  
    μe    = μ - Rf                     
    Σ_1   = inv(Σ)
    w     = Σ_1*oneV/(oneV'Σ_1*oneV)[1]
    mu   = (w'μ + (1-sum(w))*Rf)[1]
    Std = sqrt((w'Σ*w)[1])
    return w,mu,Std
end

MVMinimumVarP (generic function with 1 method)

In [19]:
(wT,muT,StdT) = MVTangencyP(μ,Σ,Rf)
(wMvp,muMvp,StdMvp) = MVMinimumVarP(μ,Σ,Rf)
println("Tangency and minum variance portfolios: ")
printmat([wT wMvp])

Tangency and minum variance portfolios: 
     0.248    -0.020
     0.682     0.625
     0.070     0.395



In [20]:
v = linspace(-2.5,2.5,101)                                 #try different mixes of wT and wMvp

(ERLev,StdLev)  = (similar(v),similar(v))
for i = 1:length(v)
    w          = v[i]*wT + (1-v[i])*wMvp                        #portfolio with v[i] in wT and 1-v[i] in wMvp
    ERLev[i]   = (w'μ)[1]
    StdLev[i]  = sqrt((w'Σ*w)[1])
end


plot(StdRp*100,mustar*100,legend=nothing,color=:red,xlims=(0,15),ylims=(0,15))
plot!(StdRpRf*100,mustar*100,legend=nothing,color=:blue)
scatter!(StdLev*100,ERLev*100)
title!("Mean vs standard deviation: scatter points are new")
xlabel!("Std(Rp), %")
ylabel!("ERp, %")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 10.0 
 
 
 12.5 
 
 
 15.0 
 
 
 Mean vs standard deviation: scatter points are new 
 
 
 Std(Rp), % 
 
 
 ERp, % 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 518.234,300.416 510.019,297.645 501.855,294.875 493.744,292.105 485.691,289.334 477.698,286.564 469.768,283.794 461.905,281.023 454.114,278.253 446.398,275.483 
 438.762,272.712 431.211,269.942 423.75,267.171 416.384,264.401 409.121,261.631 401.965,258.86 394.924,256.09 388.006,253.32 381.217,250.549 374.566,247.779 
 368.062,245.009 361.715,242.238 355.534,239.468 349.529,236.698 343.712,233.927 338.095,231.157 332.688,228.387 327.506,225.616 322.56,222.846 317.864,220.076 
 313.433,217.305 309.279,214.535 305.416,211.765 301.859,208.994 298.62,206.224 295.712,203.454 293.148,200.683 290.938,197.913 289.093,195.143 287.621,192.372 
 286.53,189.602 285.824,186.832 285.508,184.061 285.582,181.291 286.048,178.521 286.901,175.75 288.139,172.98 289.754,170.21 291.74,167.439 294.086,164.669 
 296.783,161.899 299.819,159.128 303.181,156.358 306.857,153.587 310.832,150.817 315.094,148.047 319.628,145.276 324.421,142.506 329.459,139.736 334.728,136.965 
 340.217,134.195 345.912,131.425 351.802,128.654 357.875,125.884 364.121,123.114 370.529,120.343 377.09,117.573 383.795,114.803 390.634,112.032 397.6,109.262 
 404.686,106.492 411.884,103.721 419.187,100.951 426.589,98.1807 434.085,95.4103 441.669,92.64 449.337,89.8696 457.082,87.0993 464.901,84.329 472.79,81.5586 
 480.744,78.7883 488.761,76.0179 496.837,73.2476 504.968,70.4773 513.152,67.7069 521.386,64.9366 529.667,62.1662 537.993,59.3959 546.361,56.6255 554.77,53.8552 
 563.218,51.0849 571.702,48.3145 580.22,45.5442 588.772,42.7738 597.356,40.0035 605.969,37.2332 614.612,34.4628 623.281,31.6925 631.977,28.9221 640.698,26.1518 
 649.443,23.3815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 54.7703,300.416 59.4998,297.645 64.2293,294.875 68.9587,292.105 73.6882,289.334 78.4176,286.564 83.1471,283.794 87.8766,281.023 92.606,278.253 97.3355,275.483 
 102.065,272.712 106.794,269.942 111.524,267.171 116.253,264.401 120.983,261.631 125.712,258.86 130.442,256.09 135.171,253.32 139.901,250.549 144.63,247.779 
 149.36,245.009 154.089,242.238 158.818,239.468 163.548,236.698 168.277,233.927 173.007,231.157 177.736,228.387 182.466,225.616 187.195,222.846 191.925,220.076 
 196.654,217.305 201.384,214.535 206.113,211.765 210.843,208.994 215.572,206.224 220.301,203.454 225.031,200.683 229.76,197.913 234.49,195.143 239.219,192.372 
 243.949,189.602 248.678,186.832 253.408,184.061 258.137,181.291 262.867,178.521 267.596,175.75 272.326,172.98 277.055,170.21 281.784,167.439 286.514,164.669 
 291.243,161.899 295.973,159.128 300.702,156.358 305.432,153.587 310.161,150.817 314.891,148.047 319.62,145.276 324.35,142.506 329.079,139.736 333.809,136.965 
 338.538,134.195 343.267,131.425 347.997,128.654 352.726,125.884 357.456,123.114 362.185,120.343 366.915,117.573 371.644,114.803 376.374,112.032 381.103,109.262 
 385.833,106.492 390.562,103.721 395.292,100.951 400.021,98.1807 404.751,95.4103 409.48,92.64 414.209,89.8696 418.939,87.0993 423.668,84.329 428.398,81.5586 
 433.127,78.7883 437.857,76.0179 442.586,73.2476 447.316,70.4773 452.045,67.7069 456.775,64.9366 461.504,62.1662 466.234,59.3959 470.963,56.6255 475.692,53.8552 
 480.422,51.0849 485.151,48.3145 489.881,45.5442 494.61,42.7738 499.34,40.0035 504.069,37.2332 508.799,34.4628 513.528,31.6925 518.258,28.9221 522.987,26.1518 
 527.717,23.3815 
 "/>